# Midterm project submission
Due April 4th (Sunday) 23:59pm.

You should address all the questions relevant to your project.
You will not be graded based on the values of the model performance, but on whether or not you have applied the right methodology: formulated the business model, translated it into a right machine learning approach, analyzed your data, prepared it for modelling, applied at least one machine learning approach, used cross validation for model tuning, evaluated your model using the relevant metrics.

If you have tried different approaches, please include them all, and not just the best one.
If doing some feature engineering or feature selection has improved your model, also please include all of the steps, not just the most sucessfull ones.

Here, fill out the student IDs:

## What business problem are you solving?
Please state clearly what business problem are you solving (one sentence). You may add a few sentences to explain why is this a relevant problem.


For our project we are going to try to predict if an individual will be accepted for a loan or not.

This is an extremely relevant problem specially for banks, as they have certain risk metrics that need to comply with on a daily nasis

## What is the machine learning problem that you are solving?
Is it a classification problem or a regression problem? 
Is the goal to build a model that generates a ranked list, or is it to detect anomalies as new data come in? 
Are you doing clustering to find hidden patterns?
If applicable, what is your target?

## Data exploration and preparation 

- How many data instances do you have?
- Do you have duplicates?
- How many features? What type are they?
- If they are categorical, what categories they have, what is their frequency?
- If they are numerical, what is their distribution?
- What is the distribution of the target variable?
- If you have a target, you can also check the relationship between the target and the variables.
- Do you have missing data? If yes, how are you going to handle it?
- Can you use the features in their original form, or do you need to alter them in some way?

If you have a lot of features, for midterm submission, you can choose to use only a subset.

## Feature engineering
Creating good features is probably the most important step in the machine learning process. 
This might involve doing:
- transformations
- aggregating over data points or over time and space, or finding differences (for example: differences between two monthly bills, time difference between two contacts with the client) 
- creating dummy (binary) variables
- discretization. 

Business insight is very relevant in this process.

For projects where some features are anonymized, not a lot can be done here, except preparing the format of the variables to be suitable for the selected model.


## Modeling
You should implemente AT LEAST ONE approach we covered so far, and tune of at least one hyperparameter.
Do not forget that you should split your data into train and test set.
You should do model selection and tuning using cross validation on the train set.
Explain and justify what is the metric you are using for model selection and tuning.

## Model evaluation

After selecting and tuning your model on the train set, you should evaluate its performance on the test set. 
You might have tuned your model using a certain metric, but now you should describe the model performance using all relevant metrics. 
If you have some business insight, why a certain metric is relevant, you should explain it. 
For example in disease detection, you might not want to have a false positive rate higher than some threshold (say 5%).

If you preformed clustering, you should describe and visualize your clusters.

## Next steps

If you already have an idea what are the next steps for your project, please describe them.
This part is not graded, but can help us give you feedback.